In [ ]:
import numpy as np
import torch
import os
import pickle

from Models import build
from TestTransferability import test_transferability

model_num = 1
training_switch = -1
k_max = 4
cuda = 1

if model_num == 1:
    model_name = '128 64 32 16'
    hidden_widths = [128,64,32,16]
elif model_num == 2:
    model_name = '64 32 16 8'
    hidden_widths = [64,32,16,8]
elif model_num == 3:
    model_name = '32 16 8 4'
    hidden_widths = [32,16,8,4]
elif model_num == 4:
    model_name = '128 64 32'
    hidden_widths = [128,64]
elif model_num == 5:
    model_name = '64 32 16'
    hidden_widths = [64,32,16]
elif model_num == 6:
    model_name = '32 16 8'
    hidden_widths = [32,16,8]
elif model_num == 7:
    model_name = '128 64'
    hidden_widths = [128,64]
elif model_num == 8:
    model_name = '64 32'
    hidden_widths = [64,32]
elif model_num == 9:
    model_name = '32 16'
    hidden_widths = [32,16]
elif model_num == 0:
    model_name = 'test'
    hidden_widths = [16]
else:
    raise AssertionError('Model not specified.')

if model_num == 1:
    GNN_only = False
else:
    GNN_only = True

if training_switch == -1:
    output_dir = 'unmasked'
elif training_switch == 0:
    output_dir = 'switch0'
elif training_switch == 1:
    output_dir = 'switch1'
elif training_switch == 2:
    output_dir = 'switch2'
elif training_switch == 3:
    output_dir = 'switch3'
elif training_switch == 4:
    output_dir = 'switch4'
elif training_switch == 5:
    output_dir = 'switch5'
elif training_switch == 6:
    output_dir = 'switch6'
elif training_switch == 7:
    output_dir = 'switch7'
elif training_switch == 8:
    output_dir = 'switch8'
elif training_switch == 9:
    output_dir = 'switch9'
elif training_switch == 10:
    output_dir = 'switch10'
elif training_switch == 11:
    output_dir = 'switch11'
elif training_switch == 12:
    output_dir = 'switch12'
elif training_switch == 13:
    output_dir = 'switch13'
elif training_switch == 14:
    output_dir = 'switch14'
else:
    raise AssertionError('Invalid training switch.')

graph_type = 'ErdosRenyi'
text = '_lambda_div_size'
sample_dir = os.path.dirname(os.getcwd())+'/samples/'+graph_type+text
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

all_out_dir = sample_dir.replace('samples','outputs')
if not os.path.exists(all_out_dir):
    os.makedirs(all_out_dir)
output_dir = all_out_dir + '/' + output_dir

model_dir = output_dir + '/' + model_name
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

seed_metrics = ['random']

print(model_dir)

In [ ]:
mode = 'random'

p = sample_dir.replace(graph_type+text,'real_graphs')+'/all_graphs.pkl'
if os.path.exists(p):
    with open(p, 'rb') as file:
        samples = pickle.load(file)
else:
    raise AssertionError('Data not found.')
    
for n in [25]+list(2**np.array(range(6))*50):
    for lbd in [3,4,5,6]:

        title = 'n = '+str(n)+', lambda = '+str(lbd)
        print(title)
        dir = model_dir+'/lbd '+str(lbd)
        if not os.path.exists(dir):
            os.makedirs(dir)

        flag1 = False
        flag2 = False

        result_files1 = [dir+'/GCN&SAGE_transferability_metric='+metric+'_'+str(n)+'_real_graphs.pkl' for metric in seed_metrics]
        result_files2 = [dir+'/GAT&GIN_transferability_metric='+metric+'_'+str(n)+'_real_graphs.pkl' for metric in seed_metrics]
        for p in result_files1:
            if 'harmonic' not in p and 'laplacian' not in p:
                if not os.path.exists(p):
                    flag1 = True
                    break
        for p in result_files2:
            if 'harmonic' not in p and 'laplacian' not in p:
                if not os.path.exists(p):
                    flag2 = True
                    break

        if flag1 or flag2:

            out = int(np.floor(np.sqrt(n)))

            model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'GCN','mse','adam','cyclic-cosine','reLU')
            file = model_dir+'/'+model.name+'_out'+str(out)+'_'+title+'.pth'
            if os.path.exists(file):
                model.load_state_dict(torch.load(file))
            print(model)
            model1 = model
            model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'SAGE','mse','adam','cyclic-cosine','reLU')
            file = model_dir+'/'+model.name+'_out'+str(out)+'_'+title+'.pth'
            if os.path.exists(file):
                model.load_state_dict(torch.load(file))
            print(model)
            model2 = model
            model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'GAT','mse','adam','cyclic-cosine','reLU')
            file = model_dir+'/'+model.name+'_out'+str(out)+'_'+title+'.pth'
            if os.path.exists(file):
                model.load_state_dict(torch.load(file))
            print(model)
            model3 = model
            model,criterion,optimizer,scheduler = build(out,hidden_widths,out,'GIN','mse','adam','cyclic-cosine','reLU')
            file = model_dir+'/'+model.name+'_out'+str(out)+'_'+title+'.pth'
            if os.path.exists(file):
                model.load_state_dict(torch.load(file))
            print(model)
            model4 = model
            
            result_files1 = [dir+'/GCN&SAGE_transferability_metric='+metric+'_'+str(n)+'_real_graphs_new.pkl' for metric in seed_metrics]
            result_files2 = [dir+'/GAT&GIN_transferability_metric='+metric+'_'+str(n)+'_real_graphs_new.pkl' for metric in seed_metrics]

            for p in result_files1:
                if not os.path.exists(p):
                    flag1 = True
            if flag1:
                for metric,result_file in list(zip(seed_metrics,result_files1)):
                    if metric not in ['harmonic','laplacian']:
                        print('Evaluating GCN & SAGE with '+metric+'...')
                        if not os.path.exists(result_file):
                            results = test_transferability(cuda,model1,model2,'mse',samples,seed_metric=metric,GNN_only=GNN_only)
                            with open(result_file, 'wb') as file:
                                pickle.dump(results, file)
            
            for p in result_files2:
                if not os.path.exists(p):
                    flag2 = True
            if flag2:
                for metric,result_file in list(zip(seed_metrics,result_files2)):
                    if metric not in ['harmonic','laplacian']:
                        print('Evaluating GAT & GIN with '+metric+'...')
                        if not os.path.exists(result_file):
                            results = test_transferability(cuda,model3,model4,'mse',samples,seed_metric=metric,GNN_only=True)
                            with open(result_file, 'wb') as file:
                                pickle.dump(results, file)